In [1]:
# %pip install -q langchain langchain-openai langchain-community "pydantic>=2"

In [ ]:
from enum import Enum
from typing import List, Tuple

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

try:
    
    from langchain.output_parsers import EnumOutputParser
except ImportError:
    
    from langchain.output_parsers.enum import EnumOutputParser  


try:
    from langchain.output_parsers.fix import OutputFixingParser  
except Exception:
    OutputFixingParser = None


class Sentiment(Enum):
    긍정 = "긍정"
    부정 = "부정"
    보통 = "보통"


def build_llm(model_name: str = "gpt-4o-mini", temperature: float = 0.0):
    return ChatOpenAI(model=model_name, temperature=temperature)


def build_sentiment_parser(llm=None):
    base = EnumOutputParser(enum=Sentiment)
    if OutputFixingParser and llm is not None:
        
        return OutputFixingParser.from_llm(parser=base, llm=llm)
    return base



In [4]:
def classify_review(review: str, llm=None) -> Sentiment:
    """
    한국어 영화 리뷰 텍스트를 받아 '긍정/부정/보통' 중 하나로 분류.
    """
    llm = llm or build_llm()
    parser = build_sentiment_parser(llm)
    fmt = parser.get_format_instructions()

    prompt = PromptTemplate(
        template=(
            """
            너는 한국어 영화 리뷰의 감정을 분류한다.
            가능한 레이블: 긍정, 부정, 보통 (정확히 이 셋 중 하나)
            - 부정적 표현이 강하면 '부정'
            - 칭찬/감탄/추천 뉘앙스면 '긍정'
            - 호불호가 뚜렷하지 않거나 무난하면 '보통'
            {format_instructions}

            리뷰: {review}
            """
        ),
        input_variables=["review"],
        partial_variables={"format_instructions": fmt},
    )

    chain = prompt | llm | parser
    return chain.invoke({"review": review})

def classify_many(reviews: List[str]) -> List[Tuple[str, Sentiment]]:
    llm = build_llm()
    results = []
    for r in reviews:
        try:
            lab = classify_review(r, llm=llm)
        except Exception as e:
            # 파싱/통신 예외 시 안전하게 '보통'으로 폴백 (원하면 로직 변경 가능)
            lab = Sentiment.보통
        results.append((r, lab))
    return results

def print_table(pairs: List[Tuple[str, Sentiment]]):
    print("="*12, "감정 분석 결과", "="*12)
    for text, label in pairs:
        print(f"[{label.value}] {text}")


In [5]:
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
]

results = classify_many(test_reviews)
print_table(results)


============ 감정 분석 결과 ============
[부정] 이 영화 정말 재미없어요. 시간 낭비였습니다.
[긍정] 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
[보통] 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
